In [2]:
import asyncio
import json
import logging

from signalr_async.net import Hub
from signalr_async.net.client import SignalRClient

# import merger
# from scoreboard import Scoreboard

logging.basicConfig(level=logging.DEBUG)
# s = Scoreboard()


class F1SignalRClient(SignalRClient):
    driver_list = {}
    timing_data = {}
    session_data = {}
    session_info = {}

    def LapCount(self, lc):
        pass

    def SessionData(self, session_data):
        merger.merge(self.session_data, session_data)

    def SessionInfo(self, session_info):
        merger.merge(self.session_info, session_info)

    def DriverList(self, drivers):
        merger.merge(self.driver_list, drivers)

    def TimingData(self, timing):
        merger.merge(self.timing_data, timing)

        for driver_number in timing["Lines"]:
            driver = self.driver_list[driver_number]
            driver_timing = self.timing_data["Lines"][driver_number]
            gap = ''
            color = driver['TeamColour']
            pos = driver_timing['Position']
            if not color:
                color = 'ffffff'

            if self.session_info['Type'] == "Qualifying":
                current_part = int(self.timing_data['SessionPart'])
                current_entries = self.timing_data['NoEntries'][current_part - 1]
                if driver_timing['Position'] == '1':
                    gap = driver_timing['BestLapTimes'][current_part - 1]['Value']
                elif int(driver_timing['Position']) > current_entries:
                    gap = 'KO'
                else:
                    gap = driver_timing['Stats'][current_part - 1]['TimeDiffToFastest']
            elif self.session_info['Type'] == "Practice":
                if 'TimeDiffToFastest' in driver_timing:
                    gap = driver_timing['TimeDiffToFastest']
                if driver_timing['Position'] == '1':
                    gap = driver_timing['BestLapTime']['Value']
            elif self.session_info['Type'] == "Race":
                if 'GapToLeader' in driver_timing:  # race
                    print("GapToLeader")
                    gap = driver_timing['GapToLeader']

            s.scoreboard[driver_number] = {
                "Pos": pos,
                "TLA": driver['Tla'],
                "Gap": gap,
                "Color": (int(color[:2], 16), int(color[2:4], 16), int(color[4:], 16)),
            }

    async def _process_message(self, message):
        if hasattr(message, "result"):
            print(message)
            for k in message.result:
                print(k)
                if hasattr(self, k):
                    getattr(self, k)(message.result[k])
                else:
                    print(k, message.result[k])
        elif hasattr(message, "arguments"):
            k, v, t = message.arguments
            if hasattr(self, k):
                getattr(self, k)(v)
            else:
                print(k, v)


_connection_url = 'https://livetiming.formula1.com/signalr'
hub = Hub("streaming")


async def run_client():
    async with F1SignalRClient(
            _connection_url,
            [hub],
            keepalive_interval=5,
    ) as client:
        await hub.invoke("Subscribe", ["Heartbeat", "CarData.z", "Position.z",
                       "ExtrapolatedClock", "TopThree", "RcmSeries",
                       "TimingStats", "TimingAppData",
                       "WeatherData", "TrackStatus", "DriverList",
                       "RaceControlMessages", "SessionInfo",
                       "SessionData", "LapCount", "TimingData"])
        await client.wait(timeout=5)



In [3]:
# https://github.com/iebb/PixooLiveTiming

await asyncio.gather(
        run_client(),
        # s.heartbeat()
    )


DEBUG:signalr_async.client:Starting connection with disconnected state
DEBUG:signalr_async.client:Negotiation started
DEBUG:signalr_async.client:Negotiation response: {'Url': '/signalr', 'ConnectionToken': '2ZHWgTWuQTl2HpaMYgRK1Kkh1XCwDroM187TRj25IrjsPHG7fXH0YvKQfxEZ0przaiUf8XkQsLDA/Cdf3rFwxpKUPGRVPAKY7FVlfl2Ge7P4ejRZhjGqE4QDlZuoV2py', 'ConnectionId': '62d662b4-edd9-422b-bc3d-dd7303c8d020', 'KeepAliveTimeout': 20.0, 'DisconnectTimeout': 30.0, 'ConnectionTimeout': 110.0, 'TryWebSockets': True, 'ProtocolVersion': '1.5', 'TransportConnectTimeout': 10.0, 'LongPollDelay': 1.0}
DEBUG:signalr_async.client:Connecting to wss://livetiming.formula1.com/signalr/connect?clientProtocol=1.5&connectionData=%5B%7B%22name%22:+%22streaming%22%7D%5D&transport=webSockets&connectionToken=2ZHWgTWuQTl2HpaMYgRK1Kkh1XCwDroM187TRj25IrjsPHG7fXH0YvKQfxEZ0przaiUf8XkQsLDA/Cdf3rFwxpKUPGRVPAKY7FVlfl2Ge7P4ejRZhjGqE4QDlZuoV2py
DEBUG:signalr_async.client:Initializing
DEBUG:signalr_async.client:Raw message received: WSMes

HubResult(invocation_id='0', result={'Heartbeat': {'Utc': '2023-03-18T18:07:05.4066058Z', '_kf': True}, 'CarData.z': '7ZS/CsIwEIff5eZW7k+uTbOKb6CL4iAiKEgH7Vb67tboWDR6k5LlEkI+kvwufD0s2u5yOlwhbHpYdXsIwMhSopTkl+QDuqA6Y2EWcWsoYL67jLt7oHuZH3dtezjHBYSABXCsEquDQOgK0Ofoxokfhrjpa9YZWEILbHkxma5dWWBvaZQlME4LDCOKDxAjaPkfLBOwcPMep4hbGsX1BExEVSIvli/mLDdX/bxXEaym4k49tZ7KKxX234c14q9sp8TUcJZdll2WXZbdn8uuVibVKtsu2y7bLtvu9223HW4=', 'Position.z': '7ZTPasMwDMbfRedkWH9sy77vvMF62Dp2KKOHMNaONjuVvPuSOKwuA+0FcgkE9EOfPn3WBR6P567vjgfIrxfYdJ/7c7/7/IIM5Ihbxy3qBjU7yV7uSDUlF7bQwP2hP3X7M+QL4PR56nf99/gLD4fNaff+MZY8Q27Rx9jAC+QQtYEtZCQaGiAD4cRpRtBz9AXCOEJi9dEYFkjqRugsiCPNkIYaMecR9TMiSDVjahPBmSH1NRNMJrnSh260qTmOcvEA42KbTl5bFpCb9jJpQ5Z6QYYJRZibq91Ua22TkEo9kY91A7Yg4pIa8ulmFNPmFIrNGJPUUDTlOVnkoasgNkMwRrR0knqhYqpjXwLKdTy9NxEqLrRSOxcs565taiRaFmBanijqlNNfSO134MpaMVzzOQzN/zckBE1Mab0h6w1Zb8h6Q/7ekLfhBw==', 'ExtrapolatedClock': {'Utc': '2023-03-18T18:07:16.1990405Z', 'Remaining': '00:00:00', 'Extrapolating': False, '_kf': True}, '

DEBUG:signalr_async.client:Connection keepalive
DEBUG:signalr_async.client:Sending: message_content='{}', is_binary=False
DEBUG:signalr_async.client:Connection keepalive
DEBUG:signalr_async.client:Sending: message_content='{}', is_binary=False
DEBUG:signalr_async.client:Connection keepalive
DEBUG:signalr_async.client:Sending: message_content='{}', is_binary=False
DEBUG:signalr_async.client:Connection keepalive
DEBUG:signalr_async.client:Sending: message_content='{}', is_binary=False
DEBUG:signalr_async.client:Connection keepalive
DEBUG:signalr_async.client:Sending: message_content='{}', is_binary=False
DEBUG:signalr_async.client:Connection keepalive
DEBUG:signalr_async.client:Sending: message_content='{}', is_binary=False
ERROR:signalr_async.client:Connection timeout
DEBUG:signalr_async.client:Stopping connection with connected state
DEBUG:signalr_async.client:Websocket closed
DEBUG:signalr_async.client:Session closed
DEBUG:signalr_async.client:Stopping connection with disconnected sta

NameError: name 'merger' is not defined